# n-step Temporal Difference Prediction

An implementation of "n-step TD prediction"  using a gridworld.

More info about "n-step TD prediction" can be found on section 7.1 of "Reinforcement Learning: an introduction" 2nd edition by Barto and Sutton


The gridworld has the shape(3,4) with a winning state "w"(0,3), and a lossing state "l"(1,3), a non valid state "x"(2,1) and a start state s(3,0)

|  |  |  |  |
|---|---|---|---|
|  |  |  | w |
|  |  |  | l |
|  | x |  |  |
| s |  |  |  |

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

import grid_world

### Disccount factor and step size

In [2]:

GAMMA = 0.9
ALPHA =0.1

### Auxiliary function to display the values of a policy after finishing iterative policy evaluation

In [3]:
def print_values(V,grid):
    for i in range(grid.width):
        print("--------------------------")
        for j in range(grid.height):
            v = V.get((i,j),0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="")
        print("")

### Auxiliary function to display a stochastic policy

In [4]:
def print_policy(P,grid):
    for i in range(grid.width):
        print("---------------------------")
        for j in range(grid.height):
            a = P.get((i,j),' ')
            if isinstance(a,dict):
                a = list(a)[0]
            print("  %s  |" % a, end="")
        print("")

### From or defined grid world file, import a negative grid ,retrieve all actions and states and print grid rewards
Negative grid is used to encourage the agent to find a shortest path to the goal

In [5]:
grid = grid_world.Grid.standard_grid()
states = grid.all_states()
actions = list(set([action   for action_tup in grid.actions.values() for action in action_tup]))

In [6]:
actions

['L', 'D', 'U', 'R']

In [7]:
print("Rewards of grid")
print_values(grid.rewards,grid)

Rewards of grid
--------------------------
 0.00| 0.00| 0.00| 1.00|
--------------------------
 0.00| 0.00| 0.00|-1.00|
--------------------------
 0.00| 0.00| 0.00| 0.00|


### Initialize  policy

In [8]:
policy = {(2,0):'U',
         (1,0):'U',
         (0,0):'R',
         (0,1):'R',
         (0,2):'R',
         (1,2):'R',
         (2,1):'R',
         (2,2):'R',
         (2,3):'U'}

print_policy(policy,grid)

---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  R  |     |
---------------------------
  U  |  R  |  R  |  U  |


In [9]:
def n_step_TD_prediction(policy,episodes ,gamma = 1,alpha = 0.9,n=2):
    V = defaultdict(lambda:0)
    
    for episode in range(episodes):
        s = (2,0)
        grid.set_state(s)
        T = 100000 # book code sets it to infinite
        finished =False
        t = 0
        
        rewards = dict() #to store the sequence of rewards
        states = dict() # to store the sequence of actions
        
        states[0] = s
        
        while not finished: # finished when tao = T  - 1 (update time is before end of episode)
            if t < T:
                a = policy[s]
                
                r = grid.move(a)    
                s1 = grid.current_state()
                rewards[t+1] = r
                states[t+1] = s1
                
                if grid.game_over(): #if St+1 is terminal set T to last timestemp
                    T = t + 1
                    
            updated_time =  t  - n +1 # in the book the variable is represente by "tao"
            
            if updated_time >= 0: #update only if  at least "n" steps are already performed
                
                update_limit = min(updated_time+n,T) 
                G = 0
                for i in range(updated_time +1 , update_limit + 1):
                    G+= ((gamma**(i - updated_time - 1))* rewards[i])
                    
                if updated_time + n < T:
                    G = G + ((gamma**n)*V[states[updated_time + n]])
                    
                
                    
                V[states[updated_time]]  = V[states[updated_time]] + alpha*(G -  V[states[updated_time]])
            
            s  = s1
            t += 1
                
            finished = updated_time == T -1
                
    return V

V = n_step_TD_prediction(policy,1000,0.9,0.5,3)

In [10]:
policy

{(0, 0): 'R',
 (0, 1): 'R',
 (0, 2): 'R',
 (1, 0): 'U',
 (1, 2): 'R',
 (2, 0): 'U',
 (2, 1): 'R',
 (2, 2): 'R',
 (2, 3): 'U'}

In [11]:
print("Policy")
print_policy(policy,grid)

Policy
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  R  |     |
---------------------------
  U  |  R  |  R  |  U  |


In [12]:
print_values(grid=grid,V=V)

--------------------------
 0.81| 0.90| 1.00| 0.00|
--------------------------
 0.73| 0.00| 0.00| 0.00|
--------------------------
 0.66| 0.00| 0.00| 0.00|


## Conclusions
* n-step bootstraping is a point between MC(update until end of episode) and td(0) (update with 1 immediate reward and next estimated value)
* n-step TD prediction uses n immediate rewards and the reached state after n transitions to estimate the value function
* with correct values of "n" and "alpha" it gets better results than both 1-step TD methods and monte carlo methods